In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
from data_process import TrajectoryDataset

In [43]:
from config import GenerateConfig

In [44]:
cfg = GenerateConfig(device = "cuda:0", stream_length = 20)
print(cfg)

GenerateConfig(vla_path='openvla/openvla-7b', root_dir='/mnt/sda/home/zijianwang', use_lora=True, lora_rank=48, lora_dropout=0.0, model_family='openvla', dataset_name='libero_10_no_noops', pretrained_checkpoint='/mnt/sda/home/zijianwang/openvla/FT_res/openvla-7b-finetuned-libero-10+libero_10_no_noops+b4+lr-0.0005+lora-r48+dropout-0.0--image_aug--2025-07-18_19-26-25', lora_path='/mnt/sda/home/zijianwang/openvla/adapter_tmp_dir/openvla-7b-finetuned-libero-10+libero_10_no_noops+b4+lr-0.0005+lora-r48+dropout-0.0--image_aug--2025-07-18_19-26-25', base_vla_path='/mnt/sda/home/zijianwang/HF_CACHE/openvla-7b-finetuned-libero-10', winner_trajectory_path='/mnt/sda/home/zijianwang/openvla/vla-scripts/DPO/winner_trajectory', adapter_tmp_dir='/mnt/sda/home/zijianwang/openvla/DPO_adapter_tmp_dir', run_root_dir='/mnt/sda/home/zijianwang/openvla/DPO_res', load_in_8bit=False, load_in_4bit=False, center_crop=True, batch_size=4, grad_accumulation_steps=1, learning_rate=0.0005, max_steps=10000, dpo_beta=0

In [45]:
from model_utils import setup_vla_model_with_lora, setup_model_and_config, setup_logging_and_environment

In [15]:
model = setup_vla_model_with_lora(cfg)

[*] Instantiating Pretrained VLA model
[*] Loading in BF16 with Flash-Attention Enabled


08/11 [19:54:06] WARNING  | >> Expected `transformers==4.40.1` and `tokenizers==0.19.1`   ]8;id=75954;file:///home/zijianwang/.cache/huggingface/modules/transformers_modules/openvla/openvla-7b/31f090d05236101ebfc381b61c674dd4746d4ce0/modeling_prismatic.py\modeling_prismatic.py]8;;\:]8;id=861168;file:///home/zijianwang/.cache/huggingface/modules/transformers_modules/openvla/openvla-7b/31f090d05236101ebfc381b61c674dd4746d4ce0/modeling_prismatic.py#228\228]8;;\
                          but got `transformers==4.53.3` and `tokenizers==0.21.2`; there                           
                          might be inference-time regressions due to dependency changes.                           
                          If in doubt, pleaseuse the above versions.                                               

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  8.37it/s]


In [46]:
processor, log_file, task_suite, num_tasks_in_suite, resize_size = setup_logging_and_environment(cfg, model)

Logging to local log file: ./experiments/logs/DPO-libero_10-openvla-2025_08_11-19_31_13.txt
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Task suite: libero_10


In [47]:
dataset = TrajectoryDataset(
    cfg, 
    cfg.winner_trajectory_path, 
    cfg.task_suite_name, 
    processor, 
    device=cfg.device, 
    model=model, 
    img_size=resize_size,
    stream_length=cfg.stream_length
)

Found 212 success trajectories


In [34]:
for test_idx in range(220):
    print(f"Testing with trajectory index: {test_idx}")
    # Get trajectory data
    trajectory = dataset.get_trajectory_data(test_idx)
    if len(trajectory["state"])< 10:
        print(f"Trajectory {test_idx} has less than 10 states")
        # print(trajectory)
        break
    # print(len(trajectory["state"]))


Testing with trajectory index: 0
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Testing with trajectory index: 1
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Testing with trajectory index: 2
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Testing with trajectory index: 3
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Testing with trajectory index: 4
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Testing with trajectory index: 5
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Testing with trajectory index: 6
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Testing with trajectory index: 7
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Testing with trajectory index: 8
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Testing with trajectory index: 9
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Testing with trajectory index: 10
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Testing with traject

In [40]:
traj_23 = dataset.get_trajectory_data(14)
print(len(traj_23["state"]))

[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
132


In [49]:
# Test get_initial_state multiple times to see randomness
print(f"\n6. Testing get_initial_state method (multiple runs)...")

for test_idx in range(220):
    print(f"Testing with trajectory index: {test_idx}")

    # Get trajectory data
    trajectory = dataset.get_trajectory_data(test_idx)
    print(f"\n--- Test run {test_idx+1} ---")
    
    try:
        # Call get_initial_state
        inputs, start_idx = dataset.get_initial_state(
            trajectory, 
            cfg.pretrained_checkpoint, 
            processor, 
            cfg.device
        )
        
        print(f"Start index: {start_idx}")
        # print(f"Input keys: {list(inputs.keys())}")
        
        # # Check input_ids shape and content
        # if 'input_ids' in inputs:
        #     input_ids = inputs['input_ids']
        #     print(f"Input IDs shape: {input_ids.shape}")
        #     print(f"Input IDs dtype: {input_ids.dtype}")
        #     print(f"Input IDs device: {input_ids.device}")
            
        #     # Decode first few tokens to see the prompt
        #     if hasattr(processor, 'tokenizer'):
        #         decoded = processor.tokenizer.decode(input_ids[0][:50], skip_special_tokens=False)
        #         print(f"Decoded prompt (first 50 tokens): {decoded}")
        
        # # Check pixel_values shape and content  
        # if 'pixel_values' in inputs:
        #     pixel_values = inputs['pixel_values']
        #     print(f"Pixel values shape: {pixel_values.shape}")
        #     print(f"Pixel values dtype: {pixel_values.dtype}")
        #     print(f"Pixel values device: {pixel_values.device}")
        #     print(f"Pixel values range: [{pixel_values.min():.4f}, {pixel_values.max():.4f}]")
        
        # # Check attention_mask if present
        # if 'attention_mask' in inputs:
        #     attention_mask = inputs['attention_mask']
        #     print(f"Attention mask shape: {attention_mask.shape}")
        #     print(f"Attention mask sum: {attention_mask.sum()}")
            
    except Exception as e:
        print(f"ERROR in test run {i+1}: {e}")
        import traceback
        traceback.print_exc()



6. Testing get_initial_state method (multiple runs)...
Testing with trajectory index: 0
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

--- Test run 1 ---
Start index: 240
Testing with trajectory index: 1
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

--- Test run 2 ---
Start index: 328
Testing with trajectory index: 2
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

--- Test run 3 ---
Start index: 143
Testing with trajectory index: 3
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

--- Test run 4 ---
Start index: 115
Testing with trajectory index: 4
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

--- Test run 5 ---
Start index: 37
Testing with trajectory index: 5
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

--- Test run 6 ---
Start index: 90
Testing with trajectory index: 6
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

--- Test run 7 ---
Start index: 170
Testing with trajectory index: 7
[info] using task orders [0, 1,

KeyboardInterrupt: 

In [20]:

# 7. Test with different trajectories
print(f"\n7. Testing with different trajectories...")

max_test_trajectories = min(3, len(dataset))
for idx in range(max_test_trajectories):
    print(f"\n--- Testing trajectory {idx} ---")
    try:
        trajectory = dataset.get_trajectory_data(idx)
        inputs, start_idx = dataset.get_initial_state(
            trajectory, 
            cfg.pretrained_checkpoint, 
            processor, 
            cfg.device
        )
        
        print(f"Trajectory {idx}: Task {trajectory['task_num']}, Start idx: {start_idx}, States: {len(trajectory['state'])}")
        print(f"Input shapes: input_ids={inputs['input_ids'].shape}, pixel_values={inputs['pixel_values'].shape}")
        
    except Exception as e:
        print(f"ERROR with trajectory {idx}: {e}")

print("\n" + "=" * 60)
print("Test completed!")
print("=" * 60)


7. Testing with different trajectories...

--- Testing trajectory 0 ---
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Trajectory 0: Task 2, Start idx: 6, States: 27
Input shapes: input_ids=torch.Size([1, 27]), pixel_values=torch.Size([1, 6, 224, 224])

--- Testing trajectory 1 ---
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Trajectory 1: Task 6, Start idx: 44, States: 479
Input shapes: input_ids=torch.Size([1, 37]), pixel_values=torch.Size([1, 6, 224, 224])

--- Testing trajectory 2 ---
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Trajectory 2: Task 5, Start idx: 53, States: 74
Input shapes: input_ids=torch.Size([1, 30]), pixel_values=torch.Size([1, 6, 224, 224])

Test completed!
